In [2]:
import pandas as pd
import os

In [3]:
# Set path to your raw data
data_path = "/Users/a.avira/Pet/Portfolio_Projects/Rx_Risk_Radar/data/raw"

In [4]:
print(data_path)

/Users/a.avira/Pet/Portfolio_Projects/Rx_Risk_Radar/data/raw


In [5]:
#Checking if the working directory is working properly and all the files are there
os.listdir(data_path)

['THER20Q4.txt',
 '.gitkeep',
 'DEMO20Q4.txt',
 'RPSR20Q4.txt',
 'faers_file_joining_explained.md',
 'INDI20Q4.txt',
 'DRUG20Q4.txt',
 'FDA-FAERS-Data-Dictionary.pdf',
 'OUTC20Q4.txt',
 'REAC20Q4.txt']

In [6]:
# Load the pipe-delimited FAERS demo file
# demo = pd.read_csv("DEMO20Q4.txt", sep="$", encoding='latin1')

demo = pd.read_csv(os.path.join(data_path, "DEMO20Q4.txt"), sep='$', encoding='latin1',  low_memory=False)
drug = pd.read_csv(os.path.join(data_path, "DRUG20Q4.txt"), sep='$', encoding='latin1',  low_memory=False)
reac = pd.read_csv(os.path.join(data_path, "REAC20Q4.txt"), sep='$', encoding='latin1',  low_memory=False)


In [7]:
# The FAERS .txt files are large and contain mixed data types (e.g., strings and numbers in the same column).
# Setting low_memory=False tells pandas to read the entire file into memory before inferring data types.
# This avoids DtypeWarning messages and ensures more accurate and consistent column type detection.
# low_memory=True (default)

In [8]:
# Checking what data we have in Column 21
for i, col in enumerate(demo.columns):
    print(f"{i}: {col}")

0: primaryid
1: caseid
2: caseversion
3: i_f_code
4: event_dt
5: mfr_dt
6: init_fda_dt
7: fda_dt
8: rept_cod
9: auth_num
10: mfr_num
11: mfr_sndr
12: lit_ref
13: age
14: age_cod
15: age_grp
16: sex
17: e_sub
18: wt
19: wt_cod
20: rept_dt
21: to_mfr
22: occp_cod
23: reporter_country
24: occr_country


In [9]:
# Check the shape and preview
print("Demo shape:", demo.shape)
demo.head()

Demo shape: (436148, 25)


,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,100046573,10004657,3,F,20120731.0,20201030.0,20140312,20201103,EXP,NaN,...,NaN,F,Y,81.63,KG,20201103.0,NaN,LW,US,US
1,100046962,10004696,2,F,NaN,20201019.0,20140312,20201021,EXP,NaN,...,NaN,M,Y,NaN,NaN,20201021.0,NaN,HP,PL,PL
2,100048793,10004879,3,F,20050908.0,20201030.0,20140312,20201102,EXP,NaN,...,NaN,F,Y,NaN,NaN,20201102.0,NaN,LW,US,US
3,100051383,10005138,3,F,1999.0,20201018.0,20140312,20201020,EXP,NaN,...,NaN,F,Y,83.00,KG,20201020.0,NaN,LW,US,US
4,100075524,10007552,4,F,199908.0,20201018.0,20140313,20201021,EXP,NaN,...,NaN,F,Y,90.70,KG,20201021.0,NaN,LW,US,US


In [10]:
# Check the shape and preview
print("Drug shape:", drug.shape)
drug.head()

Drug shape: (1918927, 20)


,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100046573,10004657,1,PS,LIPITOR,ATORVASTATIN CALCIUM,1,Oral,"40 MG, UNK",NaN,NaN,D,NaN,NaN,NaN,20702.0,40.0,MG,FILM-COATED TABLET,NaN
1,100046573,10004657,2,C,TOPROL XL,METOPROLOL SUCCINATE,1,NaN,UNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100046962,10004696,1,PS,QUETIAPINE.,QUETIAPINE,1,Unknown,500 MG,NaN,NaN,NaN,NaN,NaN,NaN,78679.0,500.0,MG,NaN,NaN
3,100046962,10004696,2,I,CITALOPRAM,CITALOPRAM HYDROBROMIDE,1,Unknown,10 MG,NaN,NaN,NaN,NaN,NaN,NaN,77040.0,10.0,MG,NaN,NaN
4,100046962,10004696,3,I,CITALOPRAM,CITALOPRAM HYDROBROMIDE,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77040.0,NaN,NaN,NaN,NaN


In [11]:
# Check the shape and preview
print("Reac shape:", reac.shape)
reac.head()

Reac shape: (1522657, 4)


,primaryid,caseid,pt,drug_rec_act
0,100046573,10004657,Type 2 diabetes mellitus,NaN
1,100046962,10004696,Abnormal behaviour,NaN
2,100046962,10004696,Drug interaction,NaN
3,100046962,10004696,Energy increased,NaN
4,100046962,10004696,Irritability,NaN


## 🧠 Why Join the files/tables DEMO, DRUG, and REAC?

Each FAERS file contains a part of a case:

- `DEMO`: patient info (age, sex, country)
- `DRUG`: drugs involved (names, role codes)
- `REAC`: adverse reactions reported

All are linked by `primaryid`, which represents one safety report.

Joining them gives a full view of:
- What happened (reaction)
- Who it happened to (patient)
- What drug(s) were involved

This allows:
- Analyzing side effects by demographics
- Identifying drugs with high-risk profiles
- Building visualizations for drug safety trends

In [12]:
# Quick check
print("Demo shape:", demo.shape)
print("Drug shape:", drug.shape)
print("Reac shape:", reac.shape)

Demo shape: (436148, 25)
Drug shape: (1918927, 20)
Reac shape: (1522657, 4)


<pre style="font-size:14px; font-family:monospace; line-height:1.6;">
<STRONG>We are joining 3 tables from FAERS:</STRONG>
 DEMO: Patient demographics (age, sex, etc.)
 DRUG: Drug(s) involved in the report (with roles: suspect, concomitant, etc.)
 REAC: Adverse reactions (side effects) reported
 They all use 'primaryid' as a common unique identifier for each case report.
</PRE>

#### 🔹 Step 1: Join DEMO and DRUG on 'primaryid'
#### Using INNER JOIN because we only want cases that have both patient and drug info

In [13]:
demo_drug = pd.merge(demo, drug, on = "primaryid", how = "inner")

#### 🔹 Step 2: Join the above result with REAC
#### Again using INNER JOIN to ensure we include only complete reports with a reaction listed

In [14]:
demo_drug_reac = pd.merge(demo_drug, reac, on = "primaryid", how = "inner")

<pre style="font-size:14px; font-family:monospace; line-height:1.6;">
 <STRONG>❓ Why not OUTER JOIN?</STRONG>
  OUTER JOIN would keep unmatched records, leading to:
  - rows with missing drug names or reactions
  - messy/incomplete data for analysis
  
  For analysis (like "which drugs cause which reactions?"), we only need complete records
</pre>

### 🔍 Preview the joined data

In [15]:
print("Joined shape:", demo_drug_reac.shape)
demo_drug_reac.head()

Joined shape: (11539966, 47)


,primaryid,caseid_x,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,...,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq,caseid,pt,drug_rec_act
0,100046573,10004657,3,F,20120731.0,20201030.0,20140312,20201103,EXP,NaN,...,NaN,NaN,20702.0,40.0,MG,FILM-COATED TABLET,NaN,10004657,Type 2 diabetes mellitus,NaN
1,100046573,10004657,3,F,20120731.0,20201030.0,20140312,20201103,EXP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10004657,Type 2 diabetes mellitus,NaN
2,100046962,10004696,2,F,NaN,20201019.0,20140312,20201021,EXP,NaN,...,NaN,NaN,78679.0,500.0,MG,NaN,NaN,10004696,Abnormal behaviour,NaN
3,100046962,10004696,2,F,NaN,20201019.0,20140312,20201021,EXP,NaN,...,NaN,NaN,78679.0,500.0,MG,NaN,NaN,10004696,Drug interaction,NaN
4,100046962,10004696,2,F,NaN,20201019.0,20140312,20201021,EXP,NaN,...,NaN,NaN,78679.0,500.0,MG,NaN,NaN,10004696,Energy increased,NaN


### Selecting meaningful fields for analysis and Tableau export

In [16]:
columns = [                  
    "primaryid",                                                 # unique report ID
    "caseid",                                                    # case ID (sometimes reused across reports)
    "age", "age_cod", "sex",                                     # patient details
    "drugname", "drug_seq", "role_cod",                          # drug details
    "pt"                                                         # 'preferred term' – the reported adverse reaction
]

final_df = demo_drug_reac[columns]

In [17]:
final_df.head(10)

,primaryid,caseid,age,age_cod,sex,drugname,drug_seq,role_cod,pt
0,100046573,10004657,71.0,YR,F,LIPITOR,1,PS,Type 2 diabetes mellitus
1,100046573,10004657,71.0,YR,F,TOPROL XL,2,C,Type 2 diabetes mellitus
2,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Abnormal behaviour
3,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Drug interaction
4,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Energy increased
5,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Irritability
6,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Libido increased
7,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Psychomotor hyperactivity
8,100046962,10004696,23.0,YR,M,QUETIAPINE.,1,PS,Tension
9,100046962,10004696,23.0,YR,M,CITALOPRAM,2,I,Abnormal behaviour
